# 2.1 理解词嵌入(enbedding)

深度神经网络模型，包括大型语言模型（LLMs），是无法直接处理原始文本。
由于文本属于分类数据，它与用于实施和训练神经网络的数学运算不兼容。
因此，我们需要一种方法将单词表示为连续值向量。 
（不熟悉在上下文中进行向量和张量计算的读者可以在附录A，A2.2《理解张量》中了解更多信息。）

将数据转换为向量格式的概念通常被称为嵌入(embedding)。
如图2.2所示，通过使用特定的神经网络层或其他预训练的神经网络模型，我们可以将不同类型的数据嵌入，例如视频、音频和文本。

**图2.2 
深度学习模型无法处理视频、音频和文本等原始数据格式。
因此，我们使用嵌入模型将这些原始数据转换为密集向量表示，这样深度学习架构就可以轻松理解和处理原始数据。
具体来说，此图展示了将原始数据转换为三维数值向量的过程。
需要注意的是，不同的数据格式需要不同的嵌入模型。
例如，为文本设计的嵌入模型不适用于嵌入音频或视频数据。**

![fig2.2](https://github.com/datawhalechina/llms-from-scratch-cn/blob/main/Translated_Book/img/fig-2-2.jpg?raw=true)

从本质上讲，嵌入是一种映射，它将离散对象，如单词、图像甚至整个文档，映射到连续向量空间中的点——嵌入的主要目的是将非数字数据转换为神经网络可以处理的格式。

虽然词嵌入是文本嵌入最常见的形式，但也可以用于句子、段落或整个文档的嵌入。
句子或段落嵌入是检索增强生成的流行选择。检索增强生成结合了生成（如产生文本）和检索（如搜索外部知识库）两种方式，以在生成文本时拉取相关信息，这些技术超出了本书讲述的范围。
由于我们的目标是训练类似GPT的大型语言模型（LLMs），这些模型学习一次生成一个单词的文本，因此本章着重介绍词嵌入。

目前已经开发了多种算法和框架来生成词嵌入。
其中最早和最流行的例子之一是 Word2Vec 方法。
Word2Vec 训练神经网络架构是通过预测给定目标词的上下文或反之来生成词嵌入。
Word2Vec 架构的主要思想是，出现在相似上下文中的词往往具有相似的含义。
因此，当将词嵌入投影到二维空间中以便于可视化时，可以看到相似的术语聚集在一起，如图2.3所示。

**图2.3 如果词嵌入是二维的，我们可以将它们绘制在二维散点图中以便于可视化，如此图所示。在使用词嵌入技术，例如 Word2Vec 时，对应于相似概念的词在嵌入空间中通常彼此接近。例如，在嵌入空间中，不同类型的鸟类相对于国家和城市更为靠近。**

![fig2.3](https://github.com/datawhalechina/llms-from-scratch-cn/blob/main/Translated_Book/img/fig-2-3.jpg?raw=true)

词嵌入可以有不同的维度，从一维到数千维不等。
如图2.3所示，我们可以选择二维词嵌入以便于可视化。
更高的维度可能会捕捉到词之间更多细微的关系，但作为代价，计算效率将会下降。

虽然我们可以使用预训练模型如 Word2Vec 来为机器学习模型生成嵌入，但大型语言模型（LLMs）通常会生成它们自己的嵌入，这些嵌入是输入层的一部分，并在训练期间更新。
将嵌入作为 LLM 训练的一部分进行优化，而不是使用 Word2Vec 的优势在于，嵌入被优化以适应手头的特定任务和数据。
我们将在本章后面部分实现这样的嵌入层。
此外，正如我们在第3章中讨论的，LLMs 还可以创建上下文化的输出嵌入。

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.font_manager import FontProperties
from matplotlib.patches import Rectangle, FancyBboxPatch, Circle
from IPython.display import HTML
import matplotlib.patheffects as path_effects

def create_embedding_comparison_animation():
    # 设置中文字体
    try:
        font = FontProperties(fname=r"C:\Windows\Fonts\SimHei.ttf")
    except:
        font = FontProperties(family='SimHei')
    
    # 创建图像
    plt.rcParams['figure.facecolor'] = '#f0f0f0'
    plt.rcParams['axes.facecolor'] = 'white'
    plt.rcParams['axes.grid'] = False
    
    fig, ax = plt.subplots(figsize=(16, 9))
    ax.set_facecolor('white')
    
    # 示例句子和说明
    examples = [
        {
            'sentence': "The bank approved my loan application",
            'context': "金融领域文本",
            'domain': 'financial',
            'specific_context': "贷款审批场景",
            'type': 'financial_approval'
        },
        {
            'sentence': "I sat by the river bank watching boats",
            'context': "旅游领域文本",
            'domain': 'travel',
            'specific_context': "河边游览场景",
            'type': 'geographical'
        },
        {
            'sentence': "I need to bank with a reliable institution",
            'context': "金融领域文本",
            'domain': 'financial',
            'specific_context': "选择银行场景",
            'type': 'financial_choice'
        }
    ]
    
    def draw_coordinate_system(x, y, width=3.5, height=3.5, title="", color='#3498db'):
        # 绘制标题
        title_y = y + height + 0.5
        ax.text(x + width/2, title_y, title, 
                fontsize=16, ha='center', va='center',
                fontproperties=font, color=color)
        
        # 绘制坐标系
        ax.plot([x, x + width], [y + height/2, y + height/2], 'k-', alpha=0.2)  # x轴
        ax.plot([x + width/2, x + width/2], [y, y + height], 'k-', alpha=0.2)  # y轴
        
        # 添加轴标签
        ax.text(x + width + 0.2, y + height/2, "金融维度", 
                fontsize=10, va='center', fontproperties=font, color='#666')
        ax.text(x + width/2, y + height + 0.2, "地理维度", 
                fontsize=10, ha='center', fontproperties=font, color='#666')
    
    def animate(frame):
        ax.clear()
        ax.set_xlim(0, 16)
        ax.set_ylim(0, 9)
        
        # 添加卡片背景
        card = FancyBboxPatch(
            (0.5, 0.5), 15, 8,
            boxstyle="round,pad=0.5",
            facecolor='white',
            edgecolor='#dddddd',
            linewidth=2
        )
        ax.add_patch(card)
        
        # 主标题
        title = ax.text(8, 8.2, "词嵌入方式的比较", 
                       fontsize=28, fontproperties=font,
                       ha='center', va='center', color='#2c3e50')
        title.set_path_effects([path_effects.withStroke(linewidth=3, foreground='white')])
        
        # 当前示例
        current_example = examples[frame % len(examples)]
        
        # 显示句子
        ax.text(8, 7.3, current_example['sentence'].replace('bank', 'bank'), 
                fontsize=16, ha='center', va='center', color='#34495e',
                weight='bold')
        
        ax.text(8, 6.8, f"领域：{current_example['context']} | 具体场景：{current_example['specific_context']}", 
                fontsize=14, fontproperties=font, ha='center', color='#7f8c8d')
        
        y_base = 1.8
        
        # 1. 静态词嵌入
        draw_coordinate_system(1, y_base, title="静态词嵌入 (Word2Vec)", color='#e74c3c')
        static_pos = np.array([0.5, 0.5])
        ax.scatter(1 + 1.75 + static_pos[0]*1.75, y_base + 1.75 + static_pos[1]*1.75, 
                  c='#e74c3c', s=100, zorder=5)
        ax.text(1 + 1.75 + static_pos[0]*1.75, y_base + 1.75 + static_pos[1]*1.75 + 0.2,
                "bank", ha='center', va='bottom', color='#e74c3c', fontsize=10)
        ax.text(1, y_base - 0.3, "特点：位置固定，完全忽略上下文", 
                fontsize=12, fontproperties=font, color='#7f8c8d')
        
        # 2. LLM嵌入层
        draw_coordinate_system(6, y_base, title="LLM嵌入层", color='#2ecc71')
        if current_example['domain'] == 'financial':
            llm_pos = np.array([0.7, 0.3])
        else:
            llm_pos = np.array([0.3, 0.7])
        ax.scatter(6 + 1.75 + llm_pos[0]*1.75, y_base + 1.75 + llm_pos[1]*1.75, 
                  c='#2ecc71', s=100, zorder=5)
        ax.text(6 + 1.75 + llm_pos[0]*1.75, y_base + 1.75 + llm_pos[1]*1.75 + 0.2,
                "bank", ha='center', va='bottom', color='#2ecc71', fontsize=10)
        ax.text(6, y_base - 0.3, "特点：根据训练领域优化，变化较小", 
                fontsize=12, fontproperties=font, color='#7f8c8d')
        
        # 3. 上下文化嵌入
        draw_coordinate_system(11, y_base, title="上下文化嵌入", color='#9b59b6')
        if current_example['type'] == 'geographical':
            context_pos = np.array([0.1, 0.9])
        elif current_example['type'] == 'financial_approval':
            context_pos = np.array([0.95, 0.05])
        else:  # financial_choice
            context_pos = np.array([0.8, 0.2])
        ax.scatter(11 + 1.75 + context_pos[0]*1.75, y_base + 1.75 + context_pos[1]*1.75, 
                  c='#9b59b6', s=100, zorder=5)
        ax.text(11 + 1.75 + context_pos[0]*1.75, y_base + 1.75 + context_pos[1]*1.75 + 0.2,
                "bank", ha='center', va='bottom', color='#9b59b6', fontsize=10)
        ax.text(11, y_base - 0.3, "特点：根据具体上下文实时调整，变化显著", 
                fontsize=12, fontproperties=font, color='#7f8c8d')
        
        # 在底部添加观察要点说明（始终显示）
        explanation_box = FancyBboxPatch(
            (1, 0.8), 14, 0.8,
            boxstyle="round,pad=0.3",
            facecolor='#f8f9fa',
            edgecolor='#e9ecef',
            alpha=0.8
        )
        ax.add_patch(explanation_box)
        ax.text(8, 1.1, "观察要点：LLM嵌入层主要区分'金融'vs'旅游'等大领域",
               fontsize=12, fontproperties=font, ha='center', color='#2ecc71')
        ax.text(8, 0.7, "而上下文化嵌入能识别更细微的场景差异，如'贷款审批'vs'选择银行'",
               fontsize=12, fontproperties=font, ha='center', color='#9b59b6')
        
        # 添加进度指示器
        for i in range(len(examples)):
            color = '#3498db' if i == frame % len(examples) else '#bdc3c7'
            ax.add_patch(Circle((7.5 + i*0.5, 6.3), 0.1, facecolor=color))
        
        ax.axis('off')
        return []
    
    # 创建动画
    anim = animation.FuncAnimation(
        fig, 
        animate,
        frames=len(examples) * 2,
        interval=3000,
        blit=True,
        repeat=True
    )
    
    plt.close()
    return HTML(anim.to_jshtml())

# 运行动画
create_embedding_comparison_animation()

不幸的是，高维嵌入在可视化上存在挑战，因为我们的感官感知和常见的图形表示本质上限于三维或更少，这就是为什么图2.3展示了在二维散点图中的二维嵌入)的原因。
然而，在处理大型语言模型（LLMs）时，我们通常使用的嵌入维度远高于图2.3中显示的维度。
对于GPT-2和GPT-3，嵌入大小（通常被称为模型隐藏状态的维度）根据具体的模型变种和大小而变化。
这是性能与效率之间的权衡。
最小的GPT-2（1.17亿参数）和GPT-3（1.25亿参数）模型使用768维的嵌入大小来提供具体示例。最大的GPT-3模型（1750亿参数）使用的嵌入大小为12,288维。

本章接下来的部分将介绍为大型语言模型（LLM）准备嵌入所需的那些步骤，包括将文本分割成单词、将单词转换为令牌，以及将令牌转换为嵌入向量。